In [4]:
import mysql.connector
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from pandas import Timestamp
from IPython.display import display
from Functions.connection.connection import *
from Functions.AlertP1.data_cleaning import *
from Functions.AlertP1.features import *
from Functions.analysis.step_analysis import *
from Functions.AlertP1.dummy_features import *
from Functions.Models.decision_tree import *
from Functions.Models.Logistic_regression import *
from Functions.Models.evaluation import *
import spacy
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from numpy import argsort
from Functions.AlertP1.data_cleaning import *
from Functions.AlertP1.features import *
from Functions.analysis.step_analysis import *
from Functions.AlertP1.dummy_features import *
from Functions.Models.evaluation import *
from Functions.NLP.alertp1_nlp import *
from Functions.NLP.data_with_nlp import *
from Functions.pipeline import *
from gensim.models import Word2Vec as w2v
import re
import nltk
import gensim

In [5]:
creds = ["uktuserdaroglu","Vacy4201(Pri","172.20.20.4","hgo",3306]
#Connection to the database
host = creds[2]
user = creds[0]
password = creds[1]
database = creds[3]
port = creds[4]
mydb = mysql.connector.connect(host=host, user=user, database=database, port=port, password=password, auth_plugin='mysql_native_password')
mycursor = mydb.cursor()

#Safecheck to guarantee that the connection worked
mycursor.execute('SHOW TABLES;')
print(f"Tables: {mycursor.fetchall()}")
print(mydb.connection_id) #it'll give connection_id,if got connected


Tables: [('ConsultaUrgencia_doentespedidosconsultaNeurologia2012',), ('consultaneurologia2012',), ('consultaneurologia201216anon_true',), ('hgo_data_032023',)]
955


In [6]:
alertP1 = pd.read_sql("""SELECT * FROM consultaneurologia201216anon_true""",mydb)


/Users/utkuserdaroglu/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
data= pre_process(alertP1)

/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/data_cleaning.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['PROVENIENCIA'][alertP1['PROVENIENCIA']=='']='unknown'
/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/data_cleaning.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['CTH_PRIOR'][alertP1['CTH_PRIOR']=='']='unknown'
/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/data_cleaning.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydat

In [5]:
alertP1.columns

Index(['ID_DOENTE', 'PROCESSO', 'COD_REFERENCIA', 'COD_PZ',
       'COD_UNID_SAUDE_PROV', 'UNID_PROV', 'TIPO_UNID', 'PROVENIENCIA',
       'COD_CTH_PRIOR', 'CTH_PRIOR', 'COD_MOTIVO_RECUSA', 'DES_MOTIVO_RECUSA',
       'COD_ESPECIALIDADE', 'DES_ESPECIALIDADE', 'agrupadora',
       'OUTRA_ENTIDADE', 'DATA_RECEPCAO', 'DATA_ENVIO', 'DATA_RETORNO',
       'NUM_TAXA', 'ESTADO', 'DATA_MARCACAO', 'DATA_REALIZACAO', 'OBSERVACOES',
       'Mês_entrada', 'Ano_entrada', 'trata data recusa', 'resume saída',
       'mês_saida', 'ano_saida', 'Texto', 'clean_text', 'result'],
      dtype='object')

In [5]:
#Word2Vec
import re
import nltk
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize, sent_tokenize

# Set the path to the text file
file_path = '/Users/utkuserdaroglu/Desktop/nilc_embedding_pub_corpus/chc.txt'

# Define a function to read the file and return its contents
def read_file(file_path):
    with open(file_path, 'r') as file:
        contents = file.read()
    return contents

# Define a function to tokenize a list of sentences into a list of words
def tokenize_sentences(sentences):
    tokenized_sentences = []
    for sentence in sentences:
        words = word_tokenize(sentence)
        words = [word.lower() for word in words if word.isalpha()]
        tokenized_sentences.append(words)
    return tokenized_sentences

# Read the file and tokenize its contents into a list of words
contents = read_file(file_path)
contents = re.sub(r'\s+', ' ', contents)  # Remove extra white space
sentences = sent_tokenize(contents)
tokenized_sentences = tokenize_sentences(sentences)

# Train a Word2Vec model on the tokenized sentences
model = Word2Vec(tokenized_sentences,vector_size=100, window=5, min_count=5, workers=4)




In [6]:
vocab_len = len(model.wv)
print(vocab_len)
print(list(model.wv.key_to_index))

11004
['de', 'a', 'que', 'o', 'e', 'do', 'para', 'em', 'um', 'da', 'é', 'os', 'uma', 'com', 'por', 'no', 'se', 'não', 'na', 'as', 'como', 'mais', 'mas', 'você', 'dos', 'são', 'ou', 'ao', 'já', 'das', 'muito', 'também', 'ele', 'foi', 'isso', 'até', 'sua', 'ser', 'pode', 'quando', 'seu', 'à', 'sobre', 'anos', 'era', 'eles', 'tem', 'animais', 'ainda', 'água', 'nos', 'está', 'há', 'essa', 'bem', 'só', 'ela', 'pelo', 'mesmo', 'entre', 'pela', 'brasil', 'esse', 'além', 'outros', 'conta', 'tempo', 'assim', 'cientistas', 'dia', 'seus', 'fazer', 'então', 'terra', 'onde', 'ter', 'espécies', 'vai', 'quem', 'grande', 'espécie', 'exemplo', 'depois', 'rio', 'mundo', 'cada', 'foram', 'história', 'nas', 'podem', 'hoje', 'pois', 'universidade', 'vida', 'esses', 'todos', 'sem', 'nome', 'outras', 'estão', 'nosso', 'lá', 'elas', 'explica', 'têm', 'pessoas', 'alguns', 'parte', 'apenas', 'suas', 'porque', 'vez', 'janeiro', 'pesquisadores', 'tudo', 'planeta', 'dois', 'agora', 'forma', 'meio', 'todo', 'tão', 

In [2]:
#Doc2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
from nltk.tokenize import word_tokenize

# Open the file
with open('/Users/utkuserdaroglu/Desktop/nilc_embedding_pub_corpus/chc.txt', 'r') as f:
    text = f.read()

# Tokenize the text into sentences
sentences = nltk.sent_tokenize(text)

# Tokenize the sentences into words and create TaggedDocuments
tagged_data = [TaggedDocument(words=word_tokenize(sentence.lower()), tags=[f"sentence_{i}"]) for i, sentence in enumerate(sentences)]

# Train the Doc2Vec model
model = Doc2Vec(tagged_data, vector_size=100, window=2, min_count=1, epochs=100)

# Get the vector representation of a document (the first sentence)
vector = model.infer_vector(word_tokenize(sentences[0].lower()))
print(vector)


[-0.04326106 -0.3134017  -0.18588124  0.3477289  -0.8327319   0.18695764
 -0.17792726  0.7583954   0.84245706  1.4610423   1.1902158  -0.877011
  0.4908889   0.36531195  0.97039855  0.37648135  1.1552348  -1.408344
 -0.70473194 -0.84441155  0.74312055 -0.62008244  0.49263498 -0.5564125
  0.679217    0.6200234  -0.5828578  -0.23022212 -0.72740185  0.17073108
  0.21540688 -0.01129516  0.16768177 -0.20986028  0.29157093  0.3651143
  0.3084623  -0.23307626  0.5117095   0.40574476 -0.12951535 -0.3924559
  0.15478624 -0.9699686  -0.13684091  0.67581487 -1.4532638   0.11372311
  0.2827245   0.12983094 -0.6326195  -0.37694627  0.30530873 -0.1770409
  0.3029088  -0.81266546 -0.32992277  0.12139072  0.81031996 -0.38346642
  0.28509617 -0.03020908  0.10434692 -0.14456609 -1.1182683  -0.8631322
  0.13732308  1.299657   -0.15176654 -0.66365343 -1.4863366   0.7530867
 -0.08786827 -1.1771114   1.2391822  -0.40355378  0.18043537  0.50110507
  0.0026739  -0.4162637   0.90679514 -0.42907873 -0.49000117 

In [4]:
sentences[0]

'Da próxima vez que vir sua mãe fazendo um bolo ou outra receita que utilize ovos, peça para que ela guarde as cascas para você.'